In [22]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import json

root_url = "https://meeyland.com"
base_url = "https://meeyland.com/mua-ban-nha-dat/can-ho-chung-cu-ha-noi?page="
headers = {
    "user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/74.0.3729.169 Safari/537.36",
    'referer': 'https://www.google.com/'
}


https://meeyland.com/mua-ban-nha-dat/ban-chung-cu-vi-tri-dep-toa-lac-gan-hoang-mai-ha-noi-gia-co-the-fix-1654931509548?block=danh_sach_list&filter=%7B%22need%22%3A%5B%22can_ban%22%5D%2C%22skip%22%3A46040%2C%22typeOfHouse%22%3A%5B%22chung_cu%22%5D%2C%22city%22%3A%5B%22Th%C3%A0nh%20ph%E1%BB%91%20H%C3%A0%20N%E1%BB%99i%22%5D%7D&position=46041&source=trang_danh_sach
https://meeyland.com/mua-ban-nha-dat/thao-go-no-nan-ba-n-can-ho-dt-chung-quy-71-6m2-vao-o-luon-gia-cuc-soc-chi-1-55-ty-vi-tri-hap-dan-duong-noi-ha-noi-huong-dong-bac-trong-can-ho-nay-2-phong-ngu-2-wc-ban-cong-view-dep-1650447174011?block=danh_sach_list&filter=%7B%22need%22%3A%5B%22can_ban%22%5D%2C%22skip%22%3A47840%2C%22typeOfHouse%22%3A%5B%22chung_cu%22%5D%2C%22city%22%3A%5B%22Th%C3%A0nh%20ph%E1%BB%91%20H%C3%A0%20N%E1%BB%99i%22%5D%7D&position=47841&source=trang_danh_sach
https://meeyland.com/mua-ban-nha-dat/huong-nam-ba-n-can-ho-vi-tri-dat-toa-lac-ngay-o-hai-ba-trung-ha-noi-ngoi-can-ho-nay-gom-3-phong-ngu-2-wc-khu-vuc-dan-cu-16

In [23]:
def extract_data(url):
  print(url)
  page = requests.get(url = url, headers = headers)
  soup = BeautifulSoup(page.content, 'html.parser')

  scripts = soup.find_all('script', {'type': 'application/ld+json'})

  # extract price
  gia = None
  try:
    product_script = scripts[2].text
    product_obj = json.loads(product_script)
  
    gia = product_obj['offers'][0]['price']
  except:
    print('something wrong when extract price')

  
  # extract longitude, latitude, district
  quan = None
  kinh_do = None
  vi_do = None
  try:
    place_script = scripts[3].text
    place_obj = json.loads(place_script)

    place = place_obj['name']
    place = place.split(',')
    quan = place[len(place) - 2].split('. ')[1]

    geo = place_obj['geo']
    kinh_do = geo['longitude']
    vi_do = geo['latitude']
  except:
    print('something wrong when extract place')


  # Thông tin cơ bản
  dien_tich = None
  dien_tich_su_dung = None
  huong_nha = None
  huong_ban_cong = None
  so_phong_ngu = None
  so_phong_tam = None
  giay_to = None

  basic_h5 = soup.find(lambda tag: tag.name == "h5" and 'Thông tin cơ bản' in tag.text)
  # extract dien_tich, dien_tich_su_dung
  try:
    dien_tich_title = basic_h5.find_next(lambda tag: tag.name == 'p' and 'Diện tích' in tag.text)
    dien_tich_text = dien_tich_title.find_next('p').text
    dien_tich_split = dien_tich_text.split('(')
    if(len(dien_tich_split) > 1):
      dien_tich = dien_tich_split[0]
      dien_tich_su_dung = dien_tich_split[1]
      dien_tich_su_dung = dien_tich_su_dung.replace('diện tích sử dụng ', '')
      dien_tich_su_dung = dien_tich_su_dung.replace(')', '')
    else:
      dien_tich = dien_tich_text
  except:
    print('something wrong when extract dien tich')

  # extract huong_nha
  try:
    huong_nha_title = basic_h5.find_next(lambda tag: tag.name == 'p' and 'Hướng nhà' in tag.text)
    huong_nha = huong_nha_title.find_next('p').text
  except:
    print('something wrong when extract huong nha')

  # extract huong_ban_cong
  try:
    huong_ban_cong_title = basic_h5.find_next(lambda tag: tag.name == 'p' and 'Hướng lô gia/ ban công' in tag.text)
    huong_ban_cong = huong_ban_cong_title.find_next('p').text
  except:
    print('something wrong when extract huong ban cong')

  # extract so_phong_ngu
  try:
    so_phong_ngu_title = basic_h5.find_next(lambda tag: tag.name == 'p' and 'Số phòng ngủ' in tag.text)
    so_phong_ngu = so_phong_ngu_title.find_next('p').text
  except:
    print('something wrong when extract so phong ngu')

  # extract so_phong_tam
  try:
    so_phong_tam_title = basic_h5.find_next(lambda tag: tag.name == 'p' and 'Số phòng tắm/ Toilet' in tag.text)
    so_phong_tam = so_phong_tam_title.find_next('p').text
  except:
    print('something wrong when extract so phong tam')

  # extract giay_to
  try:
    giay_to_title = basic_h5.find_next(lambda tag: tag.name == 'p' and 'Giấy tờ pháp lý' in tag.text)
    giay_to = giay_to_title.find_next('p').text
  except:
    print('something wrong when extract so phong tam')

  # Thông tin chi tiết
  duong_rong = None
  muc_do_giao_dich = None
  so_ban_cong = None

  detail_h5 = soup.find(lambda tag: tag.name == "h5" and 'Thông tin chi tiết' in tag.text)

  # extract duong_rong
  try:
    duong_rong_title = detail_h5.find_next(lambda tag: tag.name == 'p' and 'Đường rộng' in tag.text)
    duong_rong = duong_rong_title.find_next('p').text
  except:
    print('something wrong when extract duong rong')

  # extract muc_do_giao_dich
  try:
    muc_do_title = detail_h5.find_next(lambda tag: tag.name == 'p' and 'Mức độ giao dịch' in tag.text)
    muc_do_giao_dich = muc_do_title.find_next('p').text
  except:
    print('something wrong when extract muc do giao dich')

  # extract so_ban_cong
  try:
    so_ban_cong_title = detail_h5.find_next(lambda tag: tag.name == 'p' and 'Số lô gia/ ban công' in tag.text)
    so_ban_cong = so_ban_cong_title.find_next('p').text
  except:
    print('something wrong when extract so ban cong')

  # Tiện ích
  be_boi = False
  thang_may = False
  khu_de_xe = False
  san_choi = False
  san_vuon = False
  
  util_h5 = soup.find(lambda tag: tag.name == "h5" and 'Tiện ích' in tag.text)
  try:
    be_boi = util_h5.find_next(lambda tag: tag.name == 'p' and 'Bể bơi' in tag.text) != None
    thang_may = util_h5.find_next(lambda tag: tag.name == 'p' and 'Thang máy' in tag.text) != None
    khu_de_xe = util_h5.find_next(lambda tag: tag.name == 'p' and 'Khu để xe' in tag.text) != None
    san_choi = util_h5.find_next(lambda tag: tag.name == 'p' and 'Sân chơi' in tag.text) != None
    san_vuon = util_h5.find_next(lambda tag: tag.name == 'p' and 'Sân vườn' in tag.text) != None
  except:
    print('missing utils')

  # Nội thất
  noi_that = None

  noi_that_h5 = soup.find(lambda tag: tag.name == "h5" and 'Nội thất, thiết bị' in tag.text)
  if(noi_that_h5 != None):
    noi_that = noi_that_h5.find_next('div').text
  
  return (gia, quan, kinh_do, vi_do,  dien_tich, dien_tich_su_dung, huong_nha, huong_ban_cong, so_phong_ngu, so_phong_tam, giay_to, duong_rong, muc_do_giao_dich, so_ban_cong, be_boi, thang_may, khu_de_xe, san_choi, san_vuon, noi_that)


https://meeyland.com/mua-ban-nha-dat/ban-chung-cu-vi-tri-mat-tien-tan-mai-ha-noi-ban-ngay-voi-gia-chinh-chu-2-96-ty-1651734363421?block=danh_sach_list&filter=%7B%22need%22%3A%5B%22can_ban%22%5D%2C%22skip%22%3A47060%2C%22typeOfHouse%22%3A%5B%22chung_cu%22%5D%2C%22city%22%3A%5B%22Th%C3%A0nh%20ph%E1%BB%91%20H%C3%A0%20N%E1%BB%99i%22%5D%7D&position=47061&source=trang_danh_sach

https://meeyland.com/mua-ban-nha-dat/trong-can-ho-nay-thi-gom-nha-nguyen-ban-cdt-ban-can-ho-dien-tich-chung-la-80m2-nam-ngay-tren-duong-5-gia-lam-ban-ngay-voi-gia-thuong-luong-2-35-ty-1655810757583?block=danh_sach_list&filter=%7B%22need%22%3A%5B%22can_ban%22%5D%2C%22skip%22%3A44060%2C%22typeOfHouse%22%3A%5B%22chung_cu%22%5D%2C%22city%22%3A%5B%22Th%C3%A0nh%20ph%E1%BB%91%20H%C3%A0%20N%E1%BB%99i%22%5D%7D&position=44061&source=trang_danh_sach
https://meeyland.com/mua-ban-nha-dat/gia-cuc-uu-dai-2-3-ty-ba-n-can-ho-co-dt-trung-binh-76m2-vi-tri-dat-toa-lac-ngay-tai-hoang-mai-ha-noi-tong-quan-ben-trong-can-ho-2-pn-2-wc-nha-ph

In [24]:
def crawl_page(page_num):
  print('crawling page: ' + str(page_num))
  batch = []
  url = base_url + str(page_num)
  page = requests.get(url = url, headers = headers)
  soup = BeautifulSoup(page.content, 'html.parser')
  a_childs = soup.find_all('div', {'class': 'bg-white cursor-pointer slider-hover relative'})
  if a_childs == None:
    return []
  links = []
  for a_child in a_childs:
    link = a_child.find_parent('a').get('href')
    url = root_url + link
    record = extract_data(url)
    batch.append(record)
  return batch

something wrong when extract duong rongsomething wrong when extract duong rong

something wrong when extract muc do giao dichsomething wrong when extract muc do giao dich


something wrong when extract huong nhasomething wrong when extract so ban cong
something wrong when extract so ban cong

something wrong when extract huong ban cong
something wrong when extract so phong tamsomething wrong when extract huong ban cong
missing utils
https://meeyland.com/mua-ban-nha-dat/can-ho-gom-noi-that-co-ban-ban-can-ho-co-dien-tich-khoang-98m2-vi-tri-mat-tien-ngay-van-phuc-ha-dong-gia-mua-lien-tu-2-76-ty-1650456325200?block=danh_sach_list&filter=%7B%22need%22%3A%5B%22can_ban%22%5D%2C%22skip%22%3A47660%2C%22typeOfHouse%22%3A%5B%22chung_cu%22%5D%2C%22city%22%3A%5B%22Th%C3%A0nh%20ph%E1%BB%91%20H%C3%A0%20N%E1%BB%99i%22%5D%7D&position=47662&source=trang_danh_sach
something wrong when extract so phong tam
something wrong when extract so ban cong

something wrong when extract huong nhasomething wrong when

In [33]:
column_name = ['Giá', 'Quận/Huyện', 'Kinh độ', 'Vĩ độ' ,
               'Diện tích', 'Diện tích sử dụng', 'Hướng nhà', 'Hướng ban công',
               'Số phòng ngủ', 'Số phòng tắm', 'Giấy tờ', 'Đường rộng',
               'Mức độ giao dịch', 'Số ban công', 'Bể bơi', 'Thang máy', 'Khu để xe', 'Sân chơi', 'Sân vườn', 'Nội thất']
page_from = 2401
page_to = 2501

In [34]:
from concurrent.futures import ThreadPoolExecutor, as_completed
A = []
processes = []
with ThreadPoolExecutor(max_workers=200) as executor: # Tính toán đa luồng
  for i in range(page_from, page_to, 3):
    processes.append(executor.submit(lambda i: crawl_page(i) + crawl_page(i+1) + crawl_page(i+2), i)) # crawl 3 page mỗi luồng

for task in as_completed(processes):
  A += task.result()
df = pd.DataFrame.from_records(A, columns=column_name)
df

Streaming output truncated to the last 5000 lines.
something wrong when extract so phong tam
https://meeyland.com/mua-ban-nha-dat/nam-o-bac-tu-liem-ha-noi-ban-chung-cu-gia-ban-khuyen-mai-chi-2-5-ty-trong-can-ho-tong-quan-bao-gom-3-pn-2-wc-co-cho-de-xe-1650098465274?block=danh_sach_list&filter=%7B%22need%22%3A%5B%22can_ban%22%5D%2C%22skip%22%3A48500%2C%22typeOfHouse%22%3A%5B%22chung_cu%22%5D%2C%22city%22%3A%5B%22Th%C3%A0nh%20ph%E1%BB%91%20H%C3%A0%20N%E1%BB%99i%22%5D%7D&position=48519&source=trang_danh_sach
something wrong when extract so phong tamsomething wrong when extract muc do giao dich
https://meeyland.com/mua-ban-nha-dat/vi-tri-dat-gan-my-dinh-2-ha-noi-ba-n-can-ho-ban-ngay-voi-gia-thoa-thuan-tu-3-ty-trong-can-ho-nhin-chung-co-tong-3-phong-ngu-2-wc-gia-mem-sinh-vien-1650257482471?block=danh_sach_list&filter=%7B%22need%22%3A%5B%22can_ban%22%5D%2C%22skip%22%3A48100%2C%22typeOfHouse%22%3A%5B%22chung_cu%22%5D%2C%22city%22%3A%5B%22Th%C3%A0nh%20ph%E1%BB%91%20H%C3%A0%20N%E1%BB%99i%22%5D%

,Giá,Quận/Huyện,Kinh độ,Vĩ độ,Diện tích,Diện tích sử dụng,Hướng nhà,Hướng ban công,Số phòng ngủ,Số phòng tắm,Giấy tờ,Đường rộng,Mức độ giao dịch,Số ban công,Bể bơi,Thang máy,Khu để xe,Sân chơi,Sân vườn,Nội thất
0,3200000000,Hoàng Mai,105.838951,20.971995,96 m²,None,Đông Nam,None,3,2,"Giấy CN QSDĐ, Sổ đỏ, Sổ hồng",Mặt phố - Mặt đường,None,None,False,False,False,False,False,Có
1,1900000000,Long Biên,105.900643,21.026799,66 m²,None,None,None,2,None,"Giấy CN QSDĐ, Sổ đỏ, Sổ hồng",Mặt phố - Mặt đường,None,None,False,False,False,False,False,Có
2,4880000000,Nam Từ Liêm,105.778678,21.030054,108.5 m²,None,None,None,3,2,"Giấy CN QSDĐ, Sổ đỏ, Sổ hồng",Mặt phố - Mặt đường,None,None,False,False,False,False,False,Có
3,5350000000,Thanh Xuân,105.815971,21.000701,127 m²,None,Đông Bắc,None,3,2,"Giấy CN QSDĐ, Sổ đỏ, Sổ hồng",Mặt phố - Mặt đường,None,None,False,False,False,False,False,Có
4,2200000000,Gia Lâm,105.947138,20.993801,69 m²,None,None,None,2,2,"Giấy CN QSDĐ, Sổ đỏ, Sổ hồng",Mặt phố - Mặt đường,None,None,False,False,False,False,False,Có
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,1980000000,None,NaN,NaN,63.6 m²,None,Đông Bắc,None,2,1,"Giấy CN QSDĐ, Sổ đỏ, Sổ hồng",Ngõ 3 ô tô tránh,None,None,False,False,False,False,False,Có
1996,2380000000,Hoàng Mai,105.831231,20.986829,86 m²,None,Tây,None,2,2,"Giấy CN QSDĐ, Sổ đỏ, Sổ hồng",Ngõ 3 ô tô tránh,None,None,False,False,False,False,False,Có
1997,1800000000,Nam Từ Liêm,105.738917,21.003584,43.2 m²,None,Tây Nam,None,2,1,"Giấy CN QSDĐ, Sổ đỏ, Sổ hồng",Ngõ ô tô đỗ cửa,None,None,False,False,False,False,False,Có
1998,3090000000,Đống Đa,105.821965,21.002854,87 m²,None,Đông,None,2,2,"Giấy CN QSDĐ, Sổ đỏ, Sổ hồng",None,None,None,False,False,False,False,False,Có


In [35]:
from google.colab import drive
drive.mount('/content/drive')

data_path = '/content/drive/MyDrive/DS/Meeyland/'

df.to_csv(data_path + 'data-{}-{}.csv'.format(page_from, page_to))

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [38]:
files = ['data-1-101.csv', 'data-101-201.csv', 'data-201-301.csv', 'data-301-401.csv', 'data-401-501.csv', 'data-501-601.csv', 'data-601-701.csv', 'data-701-801.csv', 'data-801-901.csv', 'data-901-1001.csv', 'data-1001-1201.csv', 'data-1201-1401.csv', 'data-1401-1601.csv', 'data-1601-1801.csv', 'data-1801-2001.csv', 'data-2001-2201.csv', 'data-2201-2301.csv', 'data-2301-2401.csv', 'data-2401-2501.csv']
dfs = []
for file in files:
  dfs.append(pd.read_csv(data_path + file))

data = pd.concat(dfs, axis=0, ignore_index=True)
data.to_csv(data_path + 'data.csv')